In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime


In [4]:
user_order = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/Result_7.xlsx')

#user_cards = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/kx_users_cards.xlsx')
# column_to_drop = ['created_at','updated_at','card_number']
# user_cards.drop(columns=column_to_drop, inplace=True)


In [5]:
# ## merge with user_cards
# user_order_cards = pd.merge(user_order, user_cards, on='user_id', how='left')
# column_to_drop = ['id']
# user_order_cards.drop(columns=column_to_drop, inplace=True)

In [6]:
## add station name
stations = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/kx_stations.xlsx' )
stations2 = stations[['id','name']]
stations2.rename(columns={"id":"station_id",  "name":"station_name"}, inplace=True)
user_order_cards2 = pd.merge(user_order, stations2, on='station_id', how='left')

C:\Users\Sasan\AppData\Local\Temp\ipykernel_7332\2986992066.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations2.rename(columns={"id":"station_id",  "name":"station_name"}, inplace=True)


In [7]:
## add item name
## ino avaz kon. hamoon 2 ta table order_rows and items ro import kon va haminja merge kon
items = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/order_item.xlsx' )

user_order_cards3 = pd.merge(user_order_cards2, items, on='order_id', how='left')



In [8]:
## add is_user, year and mont, hour and uniqe_name to DF

# Convert the created_at column to datetime
user_order_cards3['created_at'] = pd.to_datetime(user_order_cards3['created_at'])

# Extract hour and minute and format as "HH:MM"
user_order_cards3['hour_minute'] = user_order_cards3['created_at'].dt.strftime('%H:%M')
# Extract year and month and format as "YYYY-MM"
user_order_cards3['year_month'] = user_order_cards3['created_at'].dt.strftime('%Y-%m')
# year_month_day
user_order_cards3['year_month_day'] = user_order_cards3['created_at'].dt.strftime('%Y-%m-%d')
# week day
user_order_cards3['weekday'] = user_order_cards3['created_at'].dt.strftime('%A')


## set not user for user_id = 0
user_order_cards3.loc[user_order_cards3['user_id'] == 0, 'username'] = 'not user'


## set is_user 1/0
user_order_cards3['is_user']=0
user_order_cards3.loc[user_order_cards3['user_id'] != 0, 'is_user'] = 1

## unique name for handling users who have several cards and users who did not register
user_order_cards3['unique_name'] = user_order_cards3['username']
user_order_cards3.loc[user_order_cards3['username'] == 'not user', 'unique_name'] = user_order_cards3['card_number']



In [16]:
## fill nan admin and merchant for user_id = 0
user_order_cards3.admin.fillna(value=0,inplace=True)
user_order_cards3.merchant.fillna(value=0,inplace=True)


##filter merchant and admin
user_order_cards3 = user_order_cards3[(user_order_cards3['merchant']==0) & (user_order_cards3['admin']==0 )]


C:\Users\Sasan\AppData\Local\Temp\ipykernel_7332\3259060630.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_order_cards3.admin.fillna(value=0,inplace=True)
C:\Users\Sasan\AppData\Local\Temp\ipykernel_7332\3259060630.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [24]:
## add frequency analysis columns : previous purchase - interval

# Convert created_at to datetime
user_order_cards3['created_at'] = pd.to_datetime(user_order_cards3['created_at'])

# Sort by unique_name and created_at
user_order_cards3 = user_order_cards3.sort_values(by=['unique_name', 'created_at'])

# Calculate the time difference between consecutive purchases
user_order_cards3['previous_purchase'] = user_order_cards3.groupby('unique_name')['created_at'].shift(1)
user_order_cards3['interval2'] = (user_order_cards3['created_at'] - user_order_cards3['previous_purchase']).dt.total_seconds() / (60*60*24)  # Convert to days

## set interval when there is sonly 2:50 distance between orders and is_error=1
user_order_cards3['interval'] = user_order_cards3['interval2']
user_order_cards3.loc[user_order_cards3['interval2']<0.002, 'interval']= np.NaN
user_order_cards3.loc[user_order_cards3['is_error']==1, 'interval']= np.NaN

user_order_cards3 = user_order_cards3.drop(columns=['interval2'])

# Compute the average interval for each customer
frequency_analysis = user_order_cards3.groupby('unique_name').agg({'interval': 'mean'}).reset_index()
frequency_analysis.rename(columns={'interval': 'average_interval_days'}, inplace=True)


user_order_cards3.to_excel('user_order_final.xlsx')


In [25]:
## group by on unique name

###is_error =1 remove for group by
## filter df
user_order_cards3 = user_order_cards3[user_order_cards3['is_error']==0]
## copy for group by on station name
user_order_cards4 = user_order_cards3.copy()

# Your initial aggregation

rfm = user_order_cards3.groupby('unique_name').aggregate({'price_total': 'count', 'year_month_day': ['min', 'max']})
rfm.columns = ['buy_count', 'date_min', 'date_max']
rfm = pd.DataFrame(rfm).reset_index()

# Group by 'unique_name', 'item_name', and 'station_name' and count occurrences
grouped_data = user_order_cards3.groupby(['unique_name', 'item_name', 'station_name']).size().reset_index(name='count')

# Identify the most frequent item_name for each unique_name
favorite_items = grouped_data.loc[grouped_data.groupby('unique_name')['count'].idxmax()][['unique_name', 'item_name', 'count']]

# Identify the most frequent station_name for each unique_name
favorite_stations = grouped_data.loc[grouped_data.groupby('unique_name')['count'].idxmax()][['unique_name', 'station_name', 'count']]

# Merge the favorite items and stations into a single DataFrame
favorite_items_and_stations = favorite_items.merge(favorite_stations, on=['unique_name', 'count'])

# Merge with rfm2 DataFrame
merged_df = rfm.merge(favorite_items_and_stations, on='unique_name', how='left')

# Merge with merged_df with frequency_analysis
merged_dff = merged_df.merge(frequency_analysis, on='unique_name', how='left')

# Create new columns for payment_getway_id counts
payment_counts = user_order_cards3.groupby(['unique_name', 'payment_getway_id']).size().unstack(fill_value=0).reset_index()
payment_counts.columns = ['unique_name', 'count_1', 'count_2', 'count_3']

# Merge with merged_df
rfm_final = merged_dff.merge(payment_counts, on='unique_name', how='left')

rfm_final.to_excel('rfm.xlsx') 


In [26]:
## add station frequency analysis columns: previous purchase - interval


# Sort by unique_name and created_at

user_order_cards4 = user_order_cards4.sort_values(by=['station_name', 'created_at'])

# Calculate the time difference between consecutive purchases
user_order_cards4['previous_purchase'] = user_order_cards4.groupby('station_name')['created_at'].shift(1)
user_order_cards4['interval2'] = (user_order_cards4['created_at'] - user_order_cards4['previous_purchase']).dt.total_seconds() / (60*60*24)  # Convert to days

## set interval when there is sonly 2:50 distance between orders
user_order_cards4['interval'] = user_order_cards4['interval2']
user_order_cards4.loc[user_order_cards4['interval2']<0.002, 'interval']= np.NaN
user_order_cards4 = user_order_cards4.drop(columns=['interval2'])

# Compute the average interval for each customer
frequency_analysis_station = user_order_cards4.groupby('station_name').agg({'interval': 'mean'}).reset_index()
frequency_analysis_station.rename(columns={'interval': 'average_interval_days'}, inplace=True)



In [27]:
## group by on unique name


# Your initial aggregation
srfm = user_order_cards4.groupby('station_name').aggregate({'price_total': 'count', 'year_month_day': ['min', 'max']})
srfm.columns = ['buy_count', 'date_min', 'date_max']
srfm = pd.DataFrame(srfm).reset_index()


# Merge with merged_df with frequency_analysis
smerged_df = srfm.merge(frequency_analysis_station, on='station_name', how='left')

# Create new columns for payment_getway_id counts
payment_counts_station = user_order_cards4.groupby(['station_name', 'payment_getway_id']).size().unstack(fill_value=0).reset_index()
payment_counts_station.columns = ['station_name', 'count_1', 'count_2', 'count_3']

# Merge with merged_df
rfm_station = smerged_df.merge(payment_counts_station, on='station_name', how='left')

rfm_station.to_excel('rfm_station.xlsx') 


In [28]:
## report to aliz : alarm!!
### check for card_numbers which have more than 1 user_id(most are firsly 0 and then creat account)
result = user_order_cards3.groupby('card_number')['user_id'].nunique()

# Filter out card numbers that refer to more than one unique user_id
unique_card_numbers = result[result > 1]
unique_card_numbers.to_excel('0cardnum_morethan1userid.xlsx')

## add is active 

In [2]:
## add is active 
import pandas as pd
import datetime as dt

rfmc = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/rfm_complete.xlsx') 
rfmsc = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/rfm_station_complete.xlsx') 
user_order = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/Book1.xlsx') 

is_active = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/isactive_stations.xlsx') 


In [11]:
rfmc2 = rfmc.merge(is_active, on='station_name', how='left')
rfmsc2 = rfmsc.merge(is_active, on='station_name', how='left')
user_order2 = user_order.merge(is_active, on='station_name', how='left')
rfmc2.to_excel('rfm_complete2.xlsx')
rfmsc2.to_excel('rfm_station_complete2.xlsx')
user_order2.to_excel('Book12.xlsx')


### finding items of high value customers

In [23]:
rfmc = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/rfm_complete.xlsx') 
rfmsc = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/rfm_station_complete.xlsx') 
user_order = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/user_order_final_isactive.xlsx') 


In [43]:
selected_rfmc = rfmc [['unique_name','RFM_Score','Segment']]
user_order2 = user_order.merge(selected_rfmc, on='unique_name', how='left')
selected_rfmsc = rfmsc [['station_name','RFM_Score_station','Segment_station']]
user_order3 = user_order2.merge(selected_rfmsc, on='station_name', how='left')

user_order3.to_excel('user_order_complete.xlsx')

Index(['Unnamed: 0', 'order_id', 'station_id', 'user_id', 'payment_getway_id',
       'price_total', 'card_number', 'created_at', 'merchant', 'admin',
       'username', 'is_reseller', 'is_error', 'mobile', 'station_name',
       'item_id', 'item_name', 'hour_minute', 'year_month', 'year_month_day',
       'weekday', 'is_user', 'unique_name', 'previous_purchase', 'interval',
       'hour', 'is_active', 'RFM_Score', 'Segment'],
      dtype='object')

In [47]:

user_order3.to_excel('user_order_complete.xlsx')